<a href="https://colab.research.google.com/github/RohanDudeja/checkout-stack/blob/main/Automating_checkout_stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💡 How to Execute the Script?
Follow these steps to execute the script and extract website URLs from an Excel column:

---
1. **Understand the Procedure**:</br>
 Begin by reviewing the provided procedure to grasp the steps involved in the process.</br>

---
2. **Provide Details**: Enter the required details for the procedure as mentioned in Step 1. ```Replace the placeholders with the actual values```</br>
 a. Set the ```inputFileName``` variable to the name of the Excel file containing the data.</br>
  b. Specify the column index with the website URLs using the ```columnIndex``` variable.</br>
  c.Specify the sheet index using ```sheetIndex``` vairable.</br>
  d. Choose an appropriate name for the output file and set it as the ```outputFileName``` variable.
---
3. **Run the Script:**</br>
Navigate to "Runtime" and select "Run all" to initiate the script's execution.

---
4. **Allow Access Popups:**</br>
Once the script starts running, you will encounter two popups requesting access permissions.</br>
a. Grant access to Google Colab for accessing Google Sheets.</br>
b. Grant access to Google Colab for accessing Google Drive. </br>
---
5. **Wait for Output:**</br>
Allow the script to process the data and extract the website URLs.
Wait for the script to complete its execution.


# 📖 **Procedure**
---
Step 1: *Set Up Variables*</br>
1. Assign the input file name to the variable ```inputFileName```.</br>
2. Determine the sheet index and assign it to vairable ```sheetIndex```.</br>
3. Determine the column index containing website URLs and assign it to the variable ```columnIndex```.</br>
> For example, if the website URL is in column A, set ```columnIndex``` to 0.</br>
> For column B, set ```columnIndex``` to 1, and so on.</br>

4. Assign the desired output file name to the variable ```outputFileName```.</br>
Make sure no file with same output name is present in your google drive.

---
Step 2: *Acess Management*</br>
Allow google colab to access drive and google sheets.

---
Step 3: *Extract details from Website URL*</br>
Loop through the selected column's cells and extract the website details.
Store the extracted details in a list or an appropriate data structure.

---
Step 4: *Write to Output File*</br>
Create the output file using the outputFileName.
Write the extracted website details to the output file.

---
Step 5: *Completion*</br>
Verify the output file in your google drive.

---

In [ ]:
# @title ▶ STEP 1
inputFileName = 'Copy of Need Checkout Stack Info'
sheetIndex = 0
columnIndex = 1
outputFileName = 'outputfile'

In [ ]:
# @title ▶ STEP 2
#attaching google sheets for file import
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
#mounting google drive for file export
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title ▶ STEP 3
#import file using file name ti pandas df
import pandas as pd
worksheet = gc.open(inputFileName).get_worksheet(sheetIndex)
rows = worksheet.get_all_values()

headers = rows[0]
rows = rows[1:]
sheetDF = pd.DataFrame.from_records(rows, columns=headers)

In [ ]:
CheckoutMap = {
    "openRzpCheckout" : "Magic",
    "gokwik-checkout": "Gokwik",
    "flo-checkout": "shopflo",
    "sr-headless-checkout": "Fastrr/Shiprocket",
    "appnova": "Appnova", #need website to verify
    "nimbbl": "Nimbbl", #need website to verify
}

PlatformMap = {
    "cdn.shopify.com": "Shopify",
    "woocommerce": "WooCommerce",
    "squarespace": "SquareSpace",
    "cdn11.bigcommerce.com": "BigCommerce",
    "Wix.com Website Builder": "Wix",
    "x-cart": "X-cart",
    "bigcartel": "Big Cartel",
    "www.weebly.com": "Weebly",
    "magento": "Magento",
    "prestashop": "PrestaShop",
    "dukaan": "Dukaan",
    "zyrosite": "Zyro"
}

In [ ]:
#defining lambda function to check website response
import requests

def crawl_website(row):
    checkout = "Not found"
    platform = "Not found"
    try:
        # Send an HTTP GET request to the URL
        print(row[columnIndex])
        response = requests.get(row[columnIndex])

        # Check if the request was successful
        if response.status_code == 200:
            # Get the HTML content of the webpage
            html = response.text
            # Search for the keywords in the HTML
            for keyword, output in CheckoutMap.items():
                if keyword in html:
                    checkout = output
                    break
            for keyword, output in PlatformMap.items():
                if keyword in html:
                    platform = output
                    break
            row['Checkout'] = checkout
            row['Platform'] = platform
            return row
    except requests.exceptions.RequestException as e:
        print("Error occurred while accessing the website:", e)

    row['Checkout'] = checkout
    row['Platform'] = platform
    return row

In [ ]:
#Process sheet using lambda function
# subsetDF = sheetDF.head(2)
subsetDF = sheetDF
subsetDF = subsetDF.apply(crawl_website, axis=1)

In [ ]:
# @title ▶ STEP 4
#Upload output
print(subsetDF)
subsetDF.to_csv('/content/drive/MyDrive/'+outputFileName+'.csv', index=False)